In [1]:
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.models import Select, CustomJS, ColumnDataSource
from bokeh.layouts import column
from bokeh.palettes import Category10
from bokeh.io import curdoc
from bokeh.models import HoverTool

In [2]:
df_ = pd.read_csv('../output_files/UCI/camas_uci_departamentos_colombia.csv')
df_['date'] = pd.to_datetime(df_['date'])

In [3]:
min(df_['date'] )

Timestamp('2020-06-01 12:00:00')

In [4]:
# Assume df_ is your main DataFrame and you have a list of unique departments and capacities
unique_departments = df_['department'].unique()
unique_capacities = df_['capacity'].unique()

# Create a dictionary to hold your ColumnDataSource objects
sources = {}

# Create a DataFrame for each combination of department and capacity
for department in unique_departments:
    for capacity in unique_capacities:
        # Create a mask for the current combination
        mask = (df_['department'] == department) & (df_['capacity'] == capacity)
        
        # Filter the DataFrame based on the mask
        filtered_df = df_[mask]
        
        # Create a ColumnDataSource from the filtered DataFrame
        sources[f'{department}_{capacity}'] = ColumnDataSource(filtered_df)


# Example DataFrame columns
columns_to_plot = ['non_covid', 'covid_suspected', 'covid_confirmed', 'available_beds', 'total_beds']
department_options = sorted(df_['department'].unique().tolist())
capacity_options = sorted(df_['capacity'].unique().tolist())


In [5]:

# Initial ColumnDataSource
initial_department = '11 - Bogotá, D.C.'
initial_capacity = 'Cuidado Intensivo Adulto'
initial_mask = (df_['department'] == initial_department) & (df_['capacity'] == initial_capacity)
initial_source = ColumnDataSource(df_[initial_mask])

# Create the figure
p = figure(x_axis_type='datetime', title="Bed Utilization Over Time", width=1000, height=400)
p.add_tools(HoverTool(tooltips=[('Date', '@date{%F}'), ('Value', '$y')],
                      formatters={'@date': 'datetime'}))

# Add lines and areas for each column
colors = Category10[len(columns_to_plot)]
for column_name, color in zip(columns_to_plot, colors):
    p.line(x='date', y=column_name, source=initial_source, line_width=2, color=color, alpha=0.8, legend_label=column_name)
    p.varea(x='date', y1=0, y2=column_name, source=initial_source, fill_color=color, fill_alpha=0.1, legend_label=column_name)

# Customize legend
p.legend.location = "top_left"
p.legend.click_policy = "hide"
# Department and Capacity Select widgets
department_select = Select(title="Department", value=initial_department, options=department_options)
capacity_select = Select(title="Capacity", value=initial_capacity, options=capacity_options)

# JavaScript callback for Select widgets
callback_code = """
    const department = department_select.value;
    const capacity = capacity_select.value;
    const key = department + '_' + capacity;
    const new_data = sources[key].data;

    // Update the data in the plot's data source
    source.data = new_data;

    // Since we've updated the data, we need to trigger a change event to update the plot
    source.change.emit();
"""

# Prepare the arguments for the CustomJS callback
callback_args = {
    'department_select': department_select,
    'capacity_select': capacity_select,
    'sources': sources,
    'source': initial_source,
    'plot': p
}

# Link the callback code to the select widgets with the correct arguments
department_select.js_on_change('value', CustomJS(args=callback_args, code=callback_code))
capacity_select.js_on_change('value', CustomJS(args=callback_args, code=callback_code))

# Layout
layout = column(department_select, capacity_select, p)

# If you are running this script in a standalone Python process, use output_file and show
output_file('../output_files/bed_utilization_plot.html')
show(layout)